# Objecttive 🎯🎯

So I wanna find out if I wanna eat at a certain time, where is the recommended venue to order the food. The consideration for whether a venue is recommended or not is how many orders the venue handles, how many item counts the venue handles, and how long the delivery time the venue requires.

The more orders a venue has handled, the more item counts it has handled, and the shorter the delivery time a venue requires, the more recommended that venue becomes

# Import Package

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings

pd.set_option('display.float_format', lambda x: '%.4f' % x)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

# Import Data

In [15]:
asli = pd.read_csv('Data/data_cluster.csv', parse_dates=['TIMESTAMP'])

In [16]:
df = asli.copy()
df.head()

,TIMESTAMP,ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES,ITEM_COUNT,USER_LAT,USER_LONG,VENUE_LAT,VENUE_LONG,ESTIMATED_DELIVERY_MINUTES,ACTUAL_DELIVERY_MINUTES,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION,CUACA
0,2020-08-01 06:07:00,-19,1,60.1580,24.9460,60.1600,24.9460,29,10,0.0000,15.0000,3.5364,0.0000,NaN
1,2020-08-01 06:17:00,-7,8,60.1630,24.9270,60.1530,24.9100,39,32,0.0000,15.0000,3.5364,0.0000,NaN
2,2020-08-01 06:54:00,-17,4,60.1610,24.9370,60.1620,24.9390,23,6,0.0000,15.0000,3.5364,0.0000,NaN
3,2020-08-01 07:09:00,-2,3,60.1850,24.9540,60.1900,24.9110,28,26,0.0000,16.7000,3.5227,0.0000,NaN
4,2020-08-01 07:10:00,-1,2,60.1820,24.9550,60.1780,24.9490,27,26,0.0000,16.7000,3.5227,0.0000,NaN


In [17]:
df['CUACA'].value_counts()

Series([], Name: CUACA, dtype: int64)

made "tanggal", "hari", and "jam" column to extracting "date" column

In [18]:
date = df['TIMESTAMP'].dt

df['tanggal'] = date.day
df['Hari'] = date.weekday
df['Jam'] = date.hour

df

,TIMESTAMP,ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES,ITEM_COUNT,USER_LAT,USER_LONG,VENUE_LAT,VENUE_LONG,ESTIMATED_DELIVERY_MINUTES,ACTUAL_DELIVERY_MINUTES,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION,CUACA,tanggal,Hari,Jam
0,2020-08-01 06:07:00,-19,1,60.1580,24.9460,60.1600,24.9460,29,10,0.0000,15.0000,3.5364,0.0000,NaN,1,5,6
1,2020-08-01 06:17:00,-7,8,60.1630,24.9270,60.1530,24.9100,39,32,0.0000,15.0000,3.5364,0.0000,NaN,1,5,6
2,2020-08-01 06:54:00,-17,4,60.1610,24.9370,60.1620,24.9390,23,6,0.0000,15.0000,3.5364,0.0000,NaN,1,5,6
3,2020-08-01 07:09:00,-2,3,60.1850,24.9540,60.1900,24.9110,28,26,0.0000,16.7000,3.5227,0.0000,NaN,1,5,7
4,2020-08-01 07:10:00,-1,2,60.1820,24.9550,60.1780,24.9490,27,26,0.0000,16.7000,3.5227,0.0000,NaN,1,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2020-09-30 19:27:00,-1,1,60.1700,24.9460,60.1700,24.9450,24,23,0.0000,12.8000,2.3442,0.0000,NaN,30,2,19
18425,2020-09-30 19:36:00,-8,6,60.1850,24.9440,60.1780,24.9490,23,15,0.0000,12.8000,2.3442,0.0000,NaN,30,2,19
18426,2020-09-30 19:39:00,-14,3,60.1860,24.9490,60.1880,24.9560,25,11,0.0000,12.8000,2.3442,0.0000,NaN,30,2,19
18427,2020-09-30 19:42:00,-11,3,60.1720,24.9290,60.1680,24.9300,21,10,0.0000,12.8000,2.3442,0.0000,NaN,30,2,19


# Calculate distance

For calculating distance between vanue and user, i trying to calculate it based on vanue longitude, vanue latitude, user longitude, and user latitude

In [19]:
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [20]:
df['distance'] = df.apply(lambda row: calculate_distance(row['USER_LAT'], row['USER_LONG'], row['VENUE_LAT'], row['VENUE_LONG']), axis=1)
df

,TIMESTAMP,ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES,ITEM_COUNT,USER_LAT,USER_LONG,VENUE_LAT,VENUE_LONG,ESTIMATED_DELIVERY_MINUTES,ACTUAL_DELIVERY_MINUTES,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION,CUACA,tanggal,Hari,Jam,distance
0,2020-08-01 06:07:00,-19,1,60.1580,24.9460,60.1600,24.9460,29,10,0.0000,15.0000,3.5364,0.0000,NaN,1,5,6,0.2224
1,2020-08-01 06:17:00,-7,8,60.1630,24.9270,60.1530,24.9100,39,32,0.0000,15.0000,3.5364,0.0000,NaN,1,5,6,1.4564
2,2020-08-01 06:54:00,-17,4,60.1610,24.9370,60.1620,24.9390,23,6,0.0000,15.0000,3.5364,0.0000,NaN,1,5,6,0.1569
3,2020-08-01 07:09:00,-2,3,60.1850,24.9540,60.1900,24.9110,28,26,0.0000,16.7000,3.5227,0.0000,NaN,1,5,7,2.4413
4,2020-08-01 07:10:00,-1,2,60.1820,24.9550,60.1780,24.9490,27,26,0.0000,16.7000,3.5227,0.0000,NaN,1,5,7,0.5549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2020-09-30 19:27:00,-1,1,60.1700,24.9460,60.1700,24.9450,24,23,0.0000,12.8000,2.3442,0.0000,NaN,30,2,19,0.0553
18425,2020-09-30 19:36:00,-8,6,60.1850,24.9440,60.1780,24.9490,23,15,0.0000,12.8000,2.3442,0.0000,NaN,30,2,19,0.8260
18426,2020-09-30 19:39:00,-14,3,60.1860,24.9490,60.1880,24.9560,25,11,0.0000,12.8000,2.3442,0.0000,NaN,30,2,19,0.4463
18427,2020-09-30 19:42:00,-11,3,60.1720,24.9290,60.1680,24.9300,21,10,0.0000,12.8000,2.3442,0.0000,NaN,30,2,19,0.4482


# Making a cool diagram

i made a HTML diagram that showing map of vanue, user and the distance between user and vanue

In [21]:
from mpl_toolkits.basemap import Basemap

# setting the size of the map
fig = plt.figure(figsize=(12,9))


# creating the map - setting latitude and longitude
m = Basemap(projection = 'mill', llcrnrlat = -90, urcrnrlat = 90, llcrnrlon = -180, urcrnrlon = 180, resolution = 'c')

<Figure size 1200x900 with 0 Axes>

In [22]:
dfPlot = df.sort_values(by='distance', ascending=False).sample(20)

In [23]:
import folium
# Create a map centered at a specific latitude and longitude
m = folium.Map(location=[60.16991980715286, 24.938722552685647], zoom_start=14)

# Iterate through DataFrame rows
# Iterate through DataFrame rows
for index, row in dfPlot.iterrows():
    customer = (row['USER_LAT'], row['USER_LONG'])
    venue = (row['VENUE_LAT'], row['VENUE_LONG'])

    # Calculate distance using Haversine formula
    distance = calculate_distance(customer[0], customer[1], venue[0], venue[1])

    # Add markers for each point
    folium.Marker(location=customer, popup=f"Customer {index+1}").add_to(m)
    folium.Marker(location=venue, popup=f"Venue {index+1}").add_to(m)

    # Draw lines between points
    folium.PolyLine(locations=[customer, venue], color='blue', popup=f"Route {index+1}<br>Distance: {distance:.2f} km<br>It takes {row['ACTUAL_DELIVERY_MINUTES']} minutes to delivered").add_to(m)

# Display the map
m.save('Visualization/mapping order.html')

## This is Cool Rightt !? 😎😎

<img src='https://cdn.discordapp.com/attachments/969207152679993414/1152128997782728704/image.png'>

# Labeling Vanue and Customer Unique

the way im doing this is i made a label based on the pair unique of longitude and latitude. 

In [24]:
df[['VENUE_LAT', 'VENUE_LONG']].value_counts()

VENUE_LAT  VENUE_LONG
60.1680    24.9300       658
60.1780    24.9490       637
60.1690    24.9330       505
           24.9410       410
60.1810    24.9250       394
                        ... 
60.1950    25.0300         1
60.1860    24.9580         1
           25.0300         1
60.1880    24.9580         1
60.2190    24.9370         1
Length: 331, dtype: int64

In [25]:
df['VANUE_lOCATION'] = df['VENUE_LAT'].astype(str) + ", " + df['VENUE_LONG'].astype(str)
df['USER_LOCATION'] = df['USER_LAT'].astype(str) + ", " + df['USER_LONG'].astype(str)

In [26]:
df['VANUE_lOCATION'].value_counts()

60.168, 24.93     658
60.178, 24.949    637
60.169, 24.933    505
60.169, 24.941    410
60.181, 24.925    394
                 ... 
60.199, 24.941      1
60.168, 24.965      1
60.151, 24.886      1
60.201, 24.882      1
60.198, 24.926      1
Name: VANUE_lOCATION, Length: 331, dtype: int64

In [27]:
unique_categories = df['VANUE_lOCATION'].unique()
mapping = {value: f'VANUE {index + 1}' for index, value in enumerate(unique_categories)}
df['VANUE'] = df['VANUE_lOCATION'].replace(mapping)
df['VANUE'].value_counts()

VANUE 10     658
VANUE 5      637
VANUE 17     505
VANUE 40     410
VANUE 57     394
            ... 
VANUE 302      1
VANUE 305      1
VANUE 306      1
VANUE 307      1
VANUE 331      1
Name: VANUE, Length: 331, dtype: int64

In [28]:
df['USER_LOCATION'].value_counts()

60.159, 24.935    47
60.161, 24.935    47
60.159, 24.937    46
60.16, 24.936     46
60.16, 24.937     43
                  ..
60.18, 24.911      1
60.174, 24.949     1
60.176, 24.931     1
60.165, 24.919     1
60.184, 24.967     1
Name: USER_LOCATION, Length: 2040, dtype: int64

In [29]:
unique_categories = df['USER_LOCATION'].unique()
mapping = {value: f'Customer {index + 1}' for index, value in enumerate(unique_categories)}
df['USER'] = df['USER_LOCATION'].replace(mapping)
df['USER'].value_counts()

Customer 221     47
Customer 429     47
Customer 369     46
Customer 272     46
Customer 335     43
                 ..
Customer 1821     1
Customer 1820     1
Customer 438      1
Customer 1583     1
Customer 2040     1
Name: USER, Length: 2040, dtype: int64

## Export the data to create dashboards in Power BI

In [31]:
df.to_csv('Data/data_cluster_olah.csv', index=False)

# Making Groupby in Order to Make a Recomendation System

the way i made a recomendation system is by groupbying data based on vanue and jam.

In [32]:
Groupby_df = df.groupby(['VANUE','Jam'])['ITEM_COUNT', 'ACTUAL_DELIVERY_MINUTES', 'distance'].mean().reset_index()
Groupby_df['SUM_OF_ORDERS'] = df.groupby(['VANUE','Jam']).size().reset_index(name = 'SUM_OF_ORDERS')['SUM_OF_ORDERS']
Groupby_df.head(20)

,VANUE,Jam,ITEM_COUNT,ACTUAL_DELIVERY_MINUTES,distance,SUM_OF_ORDERS
0,VANUE 1,5,2.0000,19.0000,0.6461,1
1,VANUE 1,6,4.5000,18.2500,0.7729,4
2,VANUE 1,7,2.0000,25.0000,0.3996,1
3,VANUE 1,8,1.8235,32.2353,0.7649,17
4,VANUE 1,9,1.3333,38.5556,0.7689,18
5,VANUE 1,10,1.1111,25.8889,0.7876,9
6,VANUE 1,11,1.8000,24.2000,0.7204,10
7,VANUE 1,12,1.5000,24.6000,0.8512,10
8,VANUE 1,13,1.5714,23.9286,1.0153,14
9,VANUE 1,14,2.0769,30.6154,0.7379,13


#### Then i scalled SUM_OF_ORDERS, distance, ACTUAL_DELIVERY_MINUTES, and iTEM_COUNT to make the data spreading from 0 and 1. 

In [33]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

scaller = MinMaxScaler()

Groupby_df['ITEM_COUNT_scalled'] = scaller.fit_transform(Groupby_df[['ITEM_COUNT']])
Groupby_df['SUM_OF_ORDERS_scalled'] = scaller.fit_transform(Groupby_df[['SUM_OF_ORDERS']])
Groupby_df['ACTUAL_DELIVERY_MINUTES_scalled'] = scaller.fit_transform(Groupby_df[['ACTUAL_DELIVERY_MINUTES']])
Groupby_df['DISTANCE_scalled'] = scaller.fit_transform(Groupby_df[['distance']])
Groupby_df

,VANUE,Jam,ITEM_COUNT,ACTUAL_DELIVERY_MINUTES,distance,SUM_OF_ORDERS,ITEM_COUNT_scalled,SUM_OF_ORDERS_scalled,ACTUAL_DELIVERY_MINUTES_scalled,DISTANCE_scalled
0,VANUE 1,5,2.0000,19.0000,0.6461,1,0.1111,0.0000,0.2391,0.1601
1,VANUE 1,6,4.5000,18.2500,0.7729,4,0.3889,0.0417,0.2228,0.1915
2,VANUE 1,7,2.0000,25.0000,0.3996,1,0.1111,0.0000,0.3696,0.0990
3,VANUE 1,8,1.8235,32.2353,0.7649,17,0.0915,0.2222,0.5269,0.1896
4,VANUE 1,9,1.3333,38.5556,0.7689,18,0.0370,0.2361,0.6643,0.1906
...,...,...,...,...,...,...,...,...,...,...
2792,VANUE 99,13,2.1250,29.1250,0.4635,8,0.1250,0.0972,0.4592,0.1149
2793,VANUE 99,14,1.8571,35.4286,0.5485,7,0.0952,0.0833,0.5963,0.1359
2794,VANUE 99,15,2.3333,35.4444,0.9150,9,0.1481,0.1111,0.5966,0.2268
2795,VANUE 99,16,3.5000,38.5000,1.1282,4,0.2778,0.0417,0.6630,0.2796


#### Next i made a item_order_weighted column based on a custom weight that i devine. i giving a bigger weight to sum of order to make it more standout than item count. i wanna make sure that when i sort the data it shows a the vanue that handle more order

In [34]:
Groupby_df['item_order_weighted'] = ((0.35 * Groupby_df['ITEM_COUNT_scalled']) +  (0.65 * Groupby_df['SUM_OF_ORDERS_scalled']))*100
Groupby_df.sort_values(by='item_order_weighted', ascending=False).head(10)

,VANUE,Jam,ITEM_COUNT,ACTUAL_DELIVERY_MINUTES,distance,SUM_OF_ORDERS,ITEM_COUNT_scalled,SUM_OF_ORDERS_scalled,ACTUAL_DELIVERY_MINUTES_scalled,DISTANCE_scalled,item_order_weighted
23,VANUE 10,15,3.8904,32.3151,0.8303,73,0.3212,1.0000,0.5286,0.2058,76.2405
26,VANUE 10,18,4.4844,27.1250,0.8107,64,0.3872,0.8750,0.4158,0.2009,70.4253
25,VANUE 10,17,4.2540,31.3810,0.8614,63,0.3616,0.8611,0.5083,0.2135,68.6265
2180,VANUE 5,16,3.6949,34.6780,0.8626,59,0.2994,0.8056,0.5800,0.2138,62.8413
21,VANUE 10,13,3.8772,28.8246,0.9570,57,0.3197,0.7778,0.4527,0.2372,61.7446
2273,VANUE 57,15,3.1429,38.8393,1.0987,56,0.2381,0.7639,0.6704,0.2723,57.9861
837,VANUE 17,15,3.2000,38.4364,1.0793,55,0.2444,0.7500,0.6617,0.2675,57.3056
2182,VANUE 5,18,3.6604,27.2642,0.9177,53,0.2956,0.7222,0.4188,0.2274,57.2904
22,VANUE 10,14,3.4074,33.2963,0.8534,54,0.2675,0.7361,0.5499,0.2115,57.2094
2181,VANUE 5,17,3.9412,33.1765,0.8807,51,0.3268,0.6944,0.5473,0.2183,56.5768


#### Then i sort the data based on 'item_order_weighted' descending, 'DISTANCE_scalled' descending, and 'ACTUAL_DELIVERY_MINUTES_scalled' ascending

In [35]:
sorted = Groupby_df.sort_values(by=['item_order_weighted', 'DISTANCE_scalled', 'ACTUAL_DELIVERY_MINUTES_scalled'], ascending=[False, False, True])

#### Finally i made a function to return the recomendation.

In [36]:
def recom(jam):
  return sorted[sorted['Jam'] == jam][['VANUE', 'Jam', 'ITEM_COUNT', 'ACTUAL_DELIVERY_MINUTES', 'SUM_OF_ORDERS','distance', 'item_order_weighted']].head(3)

recom(jam=10)

,VANUE,Jam,ITEM_COUNT,ACTUAL_DELIVERY_MINUTES,SUM_OF_ORDERS,distance,item_order_weighted
2174,VANUE 5,10,3.7451,28.9020,51,0.9255,55.8143
832,VANUE 17,10,2.0741,32.1481,54,1.0288,52.0242
2104,VANUE 44,10,2.0476,36.1667,42,1.0586,41.0880


#### I tested out the function my inputing 10. that means that i wanna find which vanue that recomend if i wanna eat at 10 o'clock. the result is the recomendation system giving a top 3 vanue that is vanue 5, vanue 17 and vanue 44.

# Snip shoot of the dashboard that i made 

<img src="https://cdn.discordapp.com/attachments/969207152679993414/1152129011372269568/image1.png">

with this dashboard i can see the mapping of customer on each vanue, how many order and customer that handled by each vanue from 1 august to 12 september 2020, and a fluctuation of item counts, and total order per hour.

<img src="https://cdn.discordapp.com/attachments/969207152679993414/1152129024966021211/image-1.png">

With this dashboard, i also can see the customer behavior. i can see the vanue favourite of the customer, how many unique vanue, how mane orders that she/he ever made, the mapping of the vanue that she/he ever order, and the fluctuation item counts, and total order per hour.

<img src="https://cdn.discordapp.com/attachments/969207152679993414/1152129038689771550/image-2.png">